## Subject-Verb-Objects

In this notebook, we conduct a series of experiments in order: 

- **First**, we isolate the subject-verb-object (SVO) triples in the texts of speakers we have gendered male or female. (Using a pandas dataframe, we save the results to s CSV files for later re-use.)
- **Second**, we compare the SVO count against the overall sentence count to determine how much of the texts have been included for analysis. See [Counts of Sentences vs SVOs](#sentences).
- **Third**, we explore usage of male and female pronouns and nouns as subjects in both corpus: first by raw count, and then by actions (verbs) associated with those nouns and pronouns. See: [Gendered Subjects](#genderedsubjects)

<div class="alert alert-block alert-info">It might be useful to find a way to combine verbs via WordNet.</div>

- **Fourth**, we map the objects associated with those actions. 

<div class="alert alert-block alert-info">The same wish, to compress a variety of words under some form of hypernym.</div>

then to explore the *character spaces* they establish for gendered entities within their speech as well as the nature of the *character space* they create for themselves as speakers. 

**Note**: We are not excluding parentheticals in this notebook.

**Next Steps**: Work on code to compile / visualize this as a network graph (?).

In [1]:
# IMPORTS
import re, spacy, textacy
import pandas as pd
from nltk import sent_tokenize

In [2]:
# Loading the Data in a gendered partitioned fashion: 
talks_m = pd.read_csv('talks_male.csv', index_col='Talk_ID')
talks_f = pd.read_csv('talks_female.csv', index_col='Talk_ID')
talks_nog = pd.read_csv('talks_nog.csv', index_col='Talk_ID')
talks_all = pd.concat([talks_m, talks_f, talks_nog])

# And then grabbing on the texts of the talks:
texts_all = talks_all.text.tolist()
texts_women = talks_f.text.tolist()
texts_men = talks_m.text.tolist()

print(f"From our {talks_all.shape[0]}x{talks_all.shape[1]} CSV, \
we have a list of {len(texts_all)} talks: {len(texts_women)} by women and \
{len(texts_men)} by men.")

From our 992x14 CSV, we have a list of 992 talks: 260 by women and 720 by men.


Lowercasing everything upfront because we don't care whether it is *She* or *she*. 

In [3]:
# Lowercase everything before we create spaCy doc and Textacy SVO triple
texts_w = [text.lower() for text in texts_women]
texts_m = [text.lower() for text in texts_men]

## 1. From Texts to SVOs

In the next section we create the SVOs that will become the basis for much of the analysis done in this notebook and the next. We first create the SVOs using spaCy's built-in functionality to do so, and then convert those SVOs to text strings that are saved in a three-column dataframe.

spaCy has three different English language models: small, medium, and large. Before proceeding, we wanted to measure the timing and production of the models available to us in spaCy. Using the the two subcorpora we have, we timed the process and counted the outcomes:

| Corpus | spaCy Model |  Time  | SVO Count |
| :-: | :-: | :-: | :-: |
| ♀︎      | small       | 1'13"  | 26,610    |
| ♀︎      | medium      | 1'19"  | 26,520    |
| ♀︎      | large       | 1'21"  | 26,527    |
| ♂︎      | small       | 3'34"  | 80,460    |
| ♂︎      | medium      | 3'56"  | 80,433    |
| ♂︎      | large       | 4'06"  | 80,331    |

According to [spaCy's documentation](https://spacy.io/models), the precision of the three models when it comes to parsing sentences is roughly equivalent, so we decide to use the smaller model for slightly faster results and a, again slightly, greater number of SVOs. (For greater readability of the comparison between the models, see this [post](https://johnlaudun.net/spacy-model-features/).)

We used the following code in a Jupyter cell to time how long it took to load the model and then generate the list of spaCy docs:

```python
%%time

nlp = spacy.load('en_core_web_lg')
docs_m = list(nlp.pipe(texts_m))
```

This uses Jupyter's now built-in timing functionality which produces the following output:

```
CPU times: user 3min 27s, sys: 33.7 s, total: 4min 1s
Wall time: 4min 6s
```

We then simply used `len(svos_m)` to get the count, but we ran it in a separate cell so that it would not affect the time of the operation above.

### 1a. Create the SVOs

After determining which model to use, we follow spaCy's conventions for feeding it a set of texts as a list of strings. 

The preview simply checks that everything went as planned: it gives us a word count and the first 50 characters -- which is weird because in theory it has converted the string to a series of spacy objects. 

In [4]:
# Load the Space pipeline to be used
nlp = spacy.load('en_core_web_sm')

# Use the pipe method to feed documents 
docs_w = list(nlp.pipe(texts_w))
docs_m = list(nlp.pipe(texts_m))

# A quick check of our work:
docs_m[0]._.preview

'Doc(2690 tokens: "  thank you so much, chris. and it\'s truly a gr...")'

### 1b. SVOs to Dataframe

Since we create SVOs for every sentence in the two subcorpora, why not save both to two dataframes?

In [5]:
def actions(doc, svo_list):
    svotriples = list(textacy.extract.triples.subject_verb_object_triples(doc))
    for item in svotriples:
        svo_list.append(
            {
                'subject': str(item[0][-1]), 
                'verb': str(item[1][-1]), 
                'object': str(item[2])
            }
        )

In [6]:
# Create the two lists
all_svos_m = []
all_svos_w = []

# Populate the lists with SVO triples
for doc in docs_m:
    actions(doc, all_svos_m)

for doc in docs_w:
    actions(doc, all_svos_w)

# Convert the lists to dataframes
svos_w = pd.DataFrame(all_svos_w)
svos_m = pd.DataFrame(all_svos_m)

print(svos_m.shape[0], svos_w.shape[0])

80460 26610


In [7]:
# Save to CSV files 
# >>> Commented out once run
# svos_w.to_csv("../output/svos_w.csv")
# svos_m.to_csv("../output/svos_m.csv")

## 2: Counts of Sentences vs SVOs <a id='sentences'></a>

The code above suggests that 70% of the SVOs in TED talks have `'i', 'we', 'she', 'he', 'they', 'it', 'you'` as their subject. It's not clear, however, how much the SVO pattern represents all sentences in the talks. In this section we explore counting sentences, both through NLTK and spaCy, but also a hand count of a few sample texts to see how well our code is reflecting underlying realities.

### NLTK

In [8]:
sents_w = [ sent_tokenize(text) for text in texts_w ]    
sents_m = [ sent_tokenize(text) for text in texts_m ]

print(len(sents_w[0]))

187


In [9]:
sent_count_m = 0
for text in texts_m:
    sent_count_m += len(sent_tokenize(text))

sent_count_w = 0
for text in texts_w:
    sent_count_w += len(sent_tokenize(text))

print(f" Female corp sent count: {sent_count_w}\n Male corp sent count: {sent_count_m}")

 Female corp sent count: 30799
 Male corp sent count: 96342


That results in the following percentages of SVOs out of the total number of sentences:

In [10]:
print(f"Female subcorpora: {svos_w.shape[0] / sent_count_w}")
print(f"Male subcorpora: {svos_m.shape[0] / sent_count_m}")

Female subcorpora: 0.8639890905548882
Male subcorpora: 0.8351497789126238


### spaCy

Our spaCy documents already exist, so we just need to use the `.sents` method to call the sentences and count them.

In [11]:
snt_cnt_w = 0
for doc in docs_w:
    snt_cnt_w += len(list(doc.sents))

snt_cnt_m = 0
for doc in docs_m:
    snt_cnt_m += len(list(doc.sents))

print(f"F: {snt_cnt_w}, M: {snt_cnt_m}.")

F: 31939, M: 99992.


In [12]:
print(f"F: {svos_w.shape[0] / snt_cnt_w}")
print(f"M: {svos_m.shape[0] / snt_cnt_m}")

F: 0.8331506935095024
M: 0.804664373149852


The total sentence counts are:
```
Women - NLTK : 30,799 with SVO ratio of 86%
        spaCy: 31,673 with SVO ratio of 84%
Men -   NLTK : 96,342 with SVO ratio of 83%
        spaCy: 99,039 with SVO ratio of 80%
```